# *합성곱 신경망*
 ## 4. 합성곱/풀링 계층 구현하기

### 1) 4차원 배열

- CNN에서 계층 사이에서 처리하는 데이터는 4차원 데이터임
- 예를 들어 형상이 (10, 1, 28, 28)인 데이터는 총 10개의 데이터(배치), 채널이 1, 높이와 너비가 28개의 원소를 보유하고 있음

In [2]:
x = np.random.rand(10, 1, 28, 28)
x.shape

(10, 1, 28, 28)

In [3]:
x[0].shape  ### indexing

(1, 28, 28)

In [7]:
x[0,0]  ### indexing

array([[7.85295580e-03, 4.40708996e-02, 8.22580975e-01, 3.14566115e-01,
        6.41721791e-01, 3.07276412e-01, 1.76985538e-01, 8.67584055e-01,
        8.63906556e-01, 3.58686740e-01, 7.26539896e-01, 9.75789876e-01,
        5.33202482e-01, 9.89016700e-01, 7.71769419e-01, 5.92085873e-01,
        7.16865990e-01, 8.10988026e-02, 2.43255981e-01, 9.61259689e-01,
        7.37279971e-01, 3.47271823e-01, 5.80641309e-01, 3.16809126e-01,
        1.83931973e-01, 7.38950175e-01, 4.67239519e-02, 3.75421177e-01],
       [3.38561270e-02, 8.74728198e-01, 5.76850252e-01, 8.57283764e-01,
        3.01875521e-01, 4.93024635e-01, 4.42627415e-02, 3.09544963e-01,
        5.53688693e-01, 4.81649899e-01, 7.27490476e-01, 5.54980045e-01,
        6.47750123e-01, 1.58237610e-01, 3.00740116e-01, 2.05033573e-01,
        9.62313669e-01, 3.08233163e-01, 2.02705138e-01, 7.80214037e-01,
        6.32113490e-01, 9.78755098e-01, 1.36247292e-01, 5.43556711e-01,
        7.32756214e-01, 8.27884421e-01, 3.13126387e-01, 3.02170

___
### 2) im2col로 데이터 전개하기

- 합성곱 연산을 구현하려면 for문이 필요
- 하지만 성능저하가 필연적인 만큼 for문 대신 **im2col(Image 2 Column)**이라는 편의 함수를 사용해 구현 시도
- im2col 함수는 입력데이터를 필터링 하기 좋게 2차원 행렬로 변환해 주는 함수임
![](image/fig 7-17.png)

<br>
- 스트라이드를 크게 잡아 필터의 적용 영역이 겹치지 않게끔 설정
![](image/fig 7-18.png)
- 하지만 실제 상황에서는 영역이 겹치는 경우가 대부분이이고, 메모리를 더 많이 소비하는 단점이 있음
- 그럼에도 불구하고 행렬계산의 편의성 때문에 이런 과정을 선호

<br>
- im2col을 이용해 합성곱 계층의 연산을 살펴보면 다음과 같은 처리과정을 수행
![](image/fig 7-19.png)
> - im2col 수행
> - 필터를 세로로 1열로 전개
>- im2col과 필터의 행렬곱 계산
>- 출력 데이터를 reshape로 차원변환

In [1]:
#import sys, os
#sys.path.append(os.pardir)
#from data.util import im2col

def im2col(input_data, filter_h, filter_w, stride=1, pad=0):
    """다수의 이미지를 입력받아 2차원 배열로 변환한다(평탄화).
    
    Parameters
    ----------
    input_data : 4차원 배열 형태의 입력 데이터(이미지 수, 채널 수, 높이, 너비)
    filter_h : 필터의 높이
    filter_w : 필터의 너비
    stride : 스트라이드
    pad : 패딩
    
    Returns
    -------
    col : 2차원 배열
    """
    N, C, H, W = input_data.shape
    out_h = (H + 2*pad - filter_h)//stride + 1
    out_w = (W + 2*pad - filter_w)//stride + 1

    img = np.pad(input_data, [(0,0), (0,0), (pad, pad), (pad, pad)], 'constant')
    col = np.zeros((N, C, filter_h, filter_w, out_h, out_w))

    for y in range(filter_h):
        y_max = y + stride*out_h
        for x in range(filter_w):
            x_max = x + stride*out_w
            col[:, :, y, x, :, :] = img[:, :, y:y_max:stride, x:x_max:stride]

    col = col.transpose(0, 4, 5, 1, 2, 3).reshape(N*out_h*out_w, -1)
    return col

- 위와 같이 im2col 구현 가능
___

In [6]:
x1 =  np.random.rand(1, 3, 7, 7)
col1 = im2col(x1, 5, 5, stride= 1, pad = 0)
col1.shape

(9, 75)

In [7]:
x2 = np.random.rand(10, 3, 7, 7)
col2 = im2col(x2, 7, 7, stride=1, pad = 0)
col2.shape

(10, 147)

>- im2col은 입력데이터의 데이터수, 채널수와 필터의 높이, 너비에 따라 출력 데이터의 차원 반환
>- $OH = \frac{H + 2P - FH}{S}+1$를 한변의 크기로 하기 때문에, 위의 예제에서 첫번째 차원의 크기는 9임 ($3^2 = (\frac{7 + 2*0 - 5}{1}+1)^2$)
>- 위의 예제에서 두번째 차원의 크기는 필터의 원소수와 같아야 하며, 따라서 75를 크기로 하는 데이터 반환 $(3*5*5)$

### 3) 합성곱 계층 구현하기

- Convolution이라는 클래스로 합성곱 계층 구현

In [3]:
class Convolution:
    def __init__(self, W, b, stride = 1, pad = 0):
        self.W = W
        self.b = b
        self.stride = stride
        self.pad = pad
        
        
    def forward(self, x):
        FN, C, FH, FW = self.W.shape
        N, C, H, W = x.shape
        out_h = int(1 + (H + 2 * self.pad - FH) / self.stride)
        #out_w = int(1 + (W + 2 * self.pad - FH) / self.stride)
        out_w = out_h
        
        col = im2col(x, FH, FW, self.stride, self.pad)   ##
        col_W = self.W.reshape(FN, -1).T    ##
        out = np.dot(col, col_W) + self.b    ##

        out = out.reshape(N, out_h, out_w, -1).transpose(0, 3, 1, 2)
        
        return out

- 합성곱 계층은 필터(가중치), 편향, 스트라이드, 패딩을 인수로 받음
- 필터는 (필터수, 채널, 높이, 너비)의 4차원 형상임
- reshape에 -1을 지정하면 뒤의 인수를 적절히 배열 (i.e. (10, 3, 5, 5) -> reshape(10, -1) ->(10, 75))
- transpose 함수는 다차원 배열의 축 순서를 변경 (i.e. (N, H, W, C) -> transpose(0, 3, 1, 2) -> (N, C, H, W))
- **Convolution Layer의 역전파도 Affine 계층의 구현과 공통점이 많으나, im2col을 역으로 처리해야 할 필요가 있음**
___

### 4) 풀링 계층 구현하기

- 풀링 계층 구현도 im2col을 활용해 입력 데이터를 전개
- 단, 풀링 적용 영역을 채널마다 독립적으로 전개
- 하단 예시 참조
![](image/fig 7-21.png)
![](image/fig 7-22.png)

- 이를 코딩으로 전개하면 아래와 같음

In [1]:
class Pooling:
    def __init__(self, pool_h, pool_w, stride = 1, pad = 0):
        self.pool_h = pool_h
        self.pool_w = pool_w
        self.stride = stride
        self.pad = pad
    
    
    def forward(self, x):
        N, C, H, W = x.shape
        out_h = int(1 + (H-self.pool_h)/self.stride)
        out_w = out_h
        #out_w = int(1 + (W-self.pool_w)/self.stride)
        
        ####전개1
        col = im2col(x, self.pool_h, self.pool_w, self.stride, self.pad)
        col = col.reshape(-1, self.pool_h * self.pool_w)
        
        ####최대값
        out = np.max(col, axis = 1)
        
        ####성형
        out = out.reshape(N, out_h, out_w, C).transpose(0, 3, 1, 2)
        
        return out
        

- 위의 그림, 코드와 같이 풀링 계층 구현은 다음 세 단계로 구현
>1. 입력 데이터를 전개
>2. 행별 최대값 계산
>3. 적절한 모형으로 성형